In [ ]:
from PIL import Image,ImageOps
import random
import os.path as osp
import os
import numpy as np
import math
import cv2
import glob
import multiprocessing
import re
import torch
import torch.nn as nn

import keras.callbacks as callbacks
import keras
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')

%matplotlib inline

## UNet with ResNet34 encoder (Pytorch)
<code>https://www.kaggle.com/rishabhiitbhu/unet-with-resnet34-encoder-pytorch/notebook\<code/>

In [ ]:
from PIL import Image,ImageOps
import random
import os.path as osp
import os
import numpy as np
import math
import cv2
import glob
import multiprocessing
import re
import torch
import torch.nn as nn

import keras.callbacks as callbacks
import keras
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
Image.MAX_IMAGE_PIXELS=None

%matplotlib inline

In [ ]:
import torch
import torch.nn as nn
from torchvision import models
train_mask_dir = "../DigestPath2019/train_mask" #create  train mask
train_dir = "../DigestPath2019/train"
def double_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        #nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, 3, padding=1),
        #nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)
    )

class BasicBlock(nn.Module):
    """Basic Block for resnet 18 and resnet 34
    """

    # BasicBlock and BottleNeck block
    # have different output size
    # we use class attribute expansion
    # to distinct
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()

        # residual function
        self.residual_function = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels * BasicBlock.expansion, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels * BasicBlock.expansion)
        )

        # shortcut
        self.shortcut = nn.Sequential()

        # the shortcut output dimension is not the same with residual function
        # use 1*1 convolution to match the dimension
        if stride != 1 or in_channels != BasicBlock.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * BasicBlock.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * BasicBlock.expansion)
            )

    def forward(self, x):
        return nn.ReLU(inplace=True)(self.residual_function(x) + self.shortcut(x))


class BottleNeck(nn.Module):
    """Residual block for resnet over 50 layers
    """
    expansion = 4

    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.residual_function = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, stride=stride, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels * BottleNeck.expansion, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels * BottleNeck.expansion),
        )

        self.shortcut = nn.Sequential()

        if stride != 1 or in_channels != out_channels * BottleNeck.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * BottleNeck.expansion, stride=stride, kernel_size=1, bias=False),
                nn.BatchNorm2d(out_channels * BottleNeck.expansion)
            )

    def forward(self, x):
        return nn.ReLU(inplace=True)(self.residual_function(x) + self.shortcut(x))



class ResNet(nn.Module):

    def __init__(self,in_channel,out_channel, block, num_block):
        super().__init__()
        #self.in_channels = in_channel
        self.outc = out_channel #不能加self.out_channel  不是很懂为啥 估计是变量优先级问题
        self.in_channels = 64
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channel, 64, kernel_size = 7, stride = 2, padding = 3,bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True))
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        # we use a different inputsize than the original paper
        # so conv2_x's stride is 1
        self.conv2_x = self._make_layer(block, 64, num_block[0], 1)
        self.conv3_x = self._make_layer(block, 128, num_block[1], 2)
        self.conv4_x = self._make_layer(block, 256, num_block[2], 2)
        self.conv5_x = self._make_layer(block, 512, num_block[3], 2)
        # self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        # self.fc = nn.Linear(512 * block.expansion, num_classes)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.dconv_up3 = double_conv(256 + 512, 256)
        self.dconv_up2 = double_conv(128 + 256, 128)
        self.dconv_up1 = double_conv(128 + 64, 64)

        self.dconv_last=nn.Sequential(
            nn.Conv2d(128, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
            nn.Conv2d(64,out_channel,1)
        )

    def _make_layer(self, block, out_channels, num_blocks, stride):
        """make resnet layers(by layer i didnt mean this 'layer' was the
        same as a neuron netowork layer, ex. conv layer), one layer may
        contain more than one residual block
        Args:
            block: block type, basic block or bottle neck block
            out_channels: output depth channel number of this layer
            num_blocks: how many blocks per layer
            stride: the stride of the first block of this layer

        Return:
            return a resnet layer
        """

        # we have num_block blocks per layer, the first block
        # could be 1 or 2, other blocks would always be 1
        strides = [stride] + [1] * (num_blocks - 1)# [stride, 1,1,1...]
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion

        return nn.Sequential(*layers)

    def forward(self, x):
        print(x.shape)
        conv1 = self.conv1(x)
        temp=self.maxpool(conv1)
        print(temp.shape)
        print(self.conv2_x)
        conv2 = self.conv2_x(temp)
        conv3 = self.conv3_x(conv2)
        conv4 = self.conv4_x(conv3)
        bottle = self.conv5_x(conv4)
        # output = self.avg_pool(output)
        # output = output.view(output.size(0), -1)
        # output = self.fc(output)
        x = self.upsample(bottle)
        # print(x.shape)
        # print(conv4.shape)
        x = torch.cat([x, conv4], dim=1)

        x = self.dconv_up3(x)
        x = self.upsample(x)
        # print(x.shape)
        # print(conv3.shape)
        x = torch.cat([x, conv3], dim=1)

        x = self.dconv_up2(x)
        x = self.upsample(x)
        # print(x.shape)
        # print(conv2.shape)
        x = torch.cat([x, conv2], dim=1)

        x = self.dconv_up1(x)
        x=self.upsample(x)
        # print(x.shape)
        # print(conv1.shape)
        x=torch.cat([x,conv1],dim=1)
        out=self.dconv_last(x)

        return out

    def load_pretrained_weights(self):
         # 导入自己模型的参数
        model_dict=self.state_dict()
        
        resnet34_weights = models.resnet34(True).state_dict()
        count_res = 0
        count_my = 0

        reskeys = list(resnet34_weights.keys())
        mykeys = list(model_dict.keys())
        # print(self)  自己网络的结构
        # print(models.resnet34())   resnet结构
        # print(reskeys)
        # print(mykeys)

        corresp_map = []
        while (True):              # 后缀相同的放入list
            reskey = reskeys[count_res]
            mykey = mykeys[count_my]

            if "fc" in reskey:
                break

            while reskey.split(".")[-1] not in mykey:
                count_my += 1
                mykey = mykeys[count_my]

            corresp_map.append([reskey, mykey])
            count_res += 1
            count_my += 1

        for k_res, k_my in corresp_map:
            model_dict[k_my]=resnet34_weights[k_res]

        try:
            self.load_state_dict(model_dict)
            print("Loaded resnet34 weights in mynet !")
        except:
            print("Error resnet34 weights in mynet !")
            raise


def resnet18():
    """ return a ResNet 18 object
    """
    return ResNet(BasicBlock, [2, 2, 2, 2])


def resnet34(in_channel,out_channel,pretrain=True):
    """ return a ResNet 34 object
    """
    model=ResNet(in_channel,out_channel,BasicBlock, [3, 4, 6, 3])
    if pretrain:
        model.load_pretrained_weights()
    return model


def resnet50():
    """ return a ResNet 50 object
    """
    return ResNet(BottleNeck, [3, 4, 6, 3])


def resnet101():
    """ return a ResNet 101 object
    """
    return ResNet(BottleNeck, [3, 4, 23, 3])


def resnet152():
    """ return a ResNet 152 object
    """
    return ResNet(BottleNeck, [3, 8, 36, 3])

if __name__ == '__main__':
    net = resnet34(3, 4, False) # out_channel = 4  4分类问题
    #print(net)
    x = torch.rand((1, 3, 512, 512)) #N，C, H, W
    print(net.forward(x).shape)


In [3]:
import argparse
import logging
import os
import sys
import model.model as model
from eval import eval_net
import numpy as np
import torch
import torch.nn as nn
from torch import optim
from PIL import Image
from tqdm import tqdm
import glob
import cv2
Image.MAX_IMAGE_PIXELS=None
%matplotlib inline

'''
from eval import eval_net
from unet import UNet
from utils.dataset import BasicDataset
'''
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, random_split, Dataset


In [5]:

def show_img(img):
    plt.figure(figsize=(18,15))
    # unnormalize
    #npimg=Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    plt.imshow(img)
    plt.show()
class BasicDataset(Dataset):
    def __init__(self, imgs_dir, masks_dir, preprocess=None,scale=0.1, mask_suffix='_mask'):      
        self.imgs_dir = imgs_dir
        self.masks_dir = masks_dir
        self.preprocess= preprocess
        #self.scale = scale
        self.scale = 512
        self.mask_suffix = mask_suffix

        self.ids = [os.path.splitext(file)[0] for file in os.listdir(imgs_dir)
                    if not file.startswith('.')]   # get prefix or so-called id
        logging.info(f'Creating dataset with {len(self.ids)} examples')
        #print("image_directory:{}  with {} files.\nmask_dir:{}".format(imgs_dir,len(self.ids),masks_dir))

    def __len__(self):
        return len(self.ids)

    @classmethod
    def process(cls, pil_img):
        #w, h = pil_img.size
        img = np.array(pil_img)
        # unnormalize
        opencvImage = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        #print("cv2 show by process")
        #cv2.imshow("sss",opencvImage)
        
        return opencvImage
        #newW, newH = int(scale * w), int(scale * h)
        #assert w > 0 and h > 0, 'Scale is too small
        #pil_img = pil_img.resize((size, size))

        ''' img_nd = np.array(pil_img)

        if len(img_nd.shape) == 2:
            img_nd = np.expand_dims(img_nd, axis=2)

        # HWC to CHW
        img_trans = img_nd.transpose((2, 0, 1))
        if img_trans.max() > 1:
            img_trans = img_trans / 255

        return img_trans'''

    def __getitem__(self, i):
        idx = self.ids[i]
        temp = os.path.join(self.masks_dir,idx+self.mask_suffix+'.*')
        mask_file = glob.glob(temp)

        img_file = glob.glob(os.path.join(self.imgs_dir , idx + '.*'))

        assert len(mask_file) == 1, \
            f'Either no mask or multiple masks found for the ID {idx}: {mask_file}'
        assert len(img_file) == 1, \
            f'Either no image or multiple images found for the ID {idx}: {img_file}'
        mask = Image.open(mask_file[0])
        #print(mask_file[0])
        img = Image.open(img_file[0])
        assert img.size == mask.size, \
            f'Image and mask {idx} should be the same size, but are {img.size} and {mask.size}'
        img = self.process(img)
        mask = self.process(mask)
        if self.preprocess!= None:
            transformed = self.preprocess(image=img,mask=mask)
            img= transformed['image']
            mask= transformed['mask']
        img_trans = img.transpose((2, 0, 1))
        if img_trans.max() > 1:
            img_trans = img_trans / 255
        #print(mask.shape)
        Grayimg = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        ret, mask_trans = cv2.threshold(Grayimg, 12, 255,cv2.THRESH_BINARY)

        
        _ = {
            'image': torch.from_numpy(img_trans).type(torch.FloatTensor),
            'mask': torch.from_numpy(mask_trans).type(torch.FloatTensor).unsqueeze(0)
        }      
        print("image final .shape",_['image'].shape)

        print("mask final .shape",_['mask'].shape)
        #show_img(img)
        return _


In [55]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from albumentations import (
    Compose, HorizontalFlip, CLAHE, HueSaturationValue,
    RandomBrightness, RandomContrast, RandomGamma,OneOf,
    ToFloat, ShiftScaleRotate,GridDistortion, ElasticTransform, JpegCompression, HueSaturationValue,
    RGBShift, RandomBrightness, RandomContrast, Blur, MotionBlur, MedianBlur, GaussNoise,CenterCrop,
    IAAAdditiveGaussianNoise,GaussNoise,OpticalDistortion,RandomSizedCrop
)
size = 512
AUGMENTATIONS_TRAIN = Compose([
    HorizontalFlip(p=0.5),
    OneOf([
        RandomContrast(),
        RandomGamma(),
        RandomBrightness()
    ], p=0.3),
    OneOf([
        ElasticTransform(alpha = 120, sigma=120*0.05,alpha_affine = 12*0.03),
        GridDistortion(),
        OpticalDistortion(distort_limit = 2, shift_limit = 0.5)
    ], p=0.3),
    RandomSizedCrop(min_max_height=(512,1024),height = size, width =size,p=1),
    ToFloat(max_value=1)
], p =1)

AUGMENTATIONS_TEST = Compose([
    ToFloat(max_value=1)
],p=1)
def show_img(img):
    plt.figure(figsize=(18,15))
    # unnormalize
    #npimg=Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    plt.imshow(img.astype(int))
    plt.show()


In [ ]:
img_url="../Colonoscopy_tissue_segment_dataset/train/1902160001_2019-06-11 12_36_32-lv1-39045-16016-3312-4096.jpg"
image = Image.open(img_url)
img = np.array(image)

print(img.size)

print(image.size)

opencvImage = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

# unnormalize
print("cv2 show by process")
#cv2.imshow("sss",opencvImage)
opencvImage=AUGMENTATIONS_TRAIN(image=opencvImage)['image']
print(opencvImage.shape)
img = opencvImage.transpose((2, 0, 1))
if img.max() > 1:
    img = img / 255 
print(img.shape)

tmp = torch.from_numpy(img).type(torch.FloatTensor)

print("tmp",tmp.shape)



In [3]:
import predict 
from train import BasicDataset
import argparse
import logging
import os
import sys
import model.model as model
from eval import eval_net
import numpy as np
import torch
import torch.nn as nn
from torch import optim
from PIL import Image
from tqdm import tqdm
import glob
import cv2
from augment import AUGMENTATIONS_TEST2 as AUGMENTATIONS_TEST
Image.MAX_IMAGE_PIXELS=None
%matplotlib inline

'''
from eval import eval_net
from unet import UNet
from utils.dataset import BasicDataset
'''
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, random_split, Dataset

import cv2
device = "cpu"
img_url = "../Colonoscopy_tissue_segment_dataset/val/18_00991B_2019-05-07 21_27_54-lv1-16174-30030-3538-5736.jpg"

load="/Users/eggwardhan/Documents/cv/Segmentation-of-colon-tumor-cells-based-on-pathological-images/check_point/nest_unet.pth"
net = model.choose_net("nested_unet")

net = net(in_channel=3,out_channel=1)
net.load_state_dict(
            torch.load(load, map_location=device)
            )

def total_predict(ori_image):
    # PIL Get image size (width, height) 
    # nparray get image size (row (height), column (width), color (3))
    img =Image.open(img_url)
    #print(img.size)
    ori_image=np.array(img)
    #ori_image = process(ori_image)
    #print("ori_image:",ori_image.shape)
    h_step = ori_image.shape[0]//256
    w_step = ori_image.shape[1]//256

    h_rest = -(ori_image.shape[0] - 256 * h_step)
    w_rest = -(ori_image.shape[1] - 256 * w_step)
    image_list = []
    predict_list = []
    for h in range(h_step):      # 截取片段
        for w in range(w_step):
            image_sample = ori_image[ (h*256):(h*256+256),
            (w*256 ) : (w*256 + 256), : ]/255
            image_list.append(image_sample)  
        image_list.append(ori_image[( h* 256) : (h*256 +256), -256:, :]/255)
    for w in range(w_step-1):   
        image_list.append(ori_image[-256:, (w*256):(w*256 +256), :]/255)
    image_list.append(ori_image[-256:, -256:, :]/255)

    for image in image_list:       
        image = image.transpose((2, 0, 1))
        image = torch.from_numpy(image).type(torch.FloatTensor)
        image = image.unsqueeze(0)
        #print(image.shape)
        image.to(device="cuda", dtype=torch.float32)
        pred1 = net(image)
        pred1 = pred1.squeeze(0).squeeze(0)  # 2 dimension
        pred = pred1>threshold
        predict_list.append(pred)
    count_temp = 0
    tmp = np.ones([ori_image.shape[0],ori_image.shape[1]])
    for h in range(h_step):
        for w in range(w_step):
            tmp[
                h*256:(h+1)*256,
                w*256:(w+1)*256
            ] = predict_list[count_temp]
            count_temp += 1
        tmp[h *256 :(h+1) *256, w_rest:] = predict_list[count_temp][:, w_rest:]
        count_temp+=1
    for w in range(w_step -1):
        tmp[h_rest:, (w *256):(w*256+256)] = predict_list[count_temp][h_rest:,:]
        count_temp+=1
    tmp[-257:-1,-257:-1] = predict_list[count_temp][:, :]

pred= total_predict(Image.open(img_url))
#print(type(pred))
#print(pred)
show_img(pred)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/eggwardhan/Documents/cv/Segmentation-of-colon-tumor-cells-based-on-pathological-images/check_point/nest_unet.pth'

In [ ]:
# -*- coding: utf-8 -*-

# version 1 mile
# refer : https://github.com/milesial/Pytorch-UNet/blob/master/train.py
import argparse
import logging
import os
import cv2
import sys
import cv2
import model.model as model
from eval import eval_net
import numpy as np
import torch
import torch.nn as nn
from torch import optim
from PIL import Image
from tqdm import tqdm
import glob
from augment import AUGMENTATIONS_TRAIN,AUGMENTATIONS_TEST
Image.MAX_IMAGE_PIXELS=None

'''
from eval import eval_net
from unet import UNet
from utils.dataset import BasicDataset
'''
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, random_split, Dataset

dir_checkpoint= "./check_point/"

train_mask_dir = "../Colonoscopy_tissue_segment_dataset/train_mask" #create  train mask
train_dir = "../Colonoscopy_tissue_segment_dataset/train" # create train data
val_mask_dir = "../Colonoscopy_tissue_segment_dataset/val_mask"
val_dir = "../Colonoscopy_tissue_segment_dataset/val"


class BasicDataset(Dataset):
    def __init__(self, imgs_dir, masks_dir, preprocess=None, scale=512, mask_suffix='_mask'):
        self.imgs_dir = imgs_dir
        self.masks_dir = masks_dir
        #self.scale = scale
        self.preprocess= preprocess
        self.scale = scale
        self.mask_suffix = mask_suffix
        self.ids = [os.path.splitext(file)[0] for file in os.listdir(imgs_dir)
                    if not file.startswith('.')]   # get prefix or so-called id
        logging.info(f'Creating dataset with {len(self.ids)} examples')
        #print("image_directory:{}  with {} files.\nmask_dir:{}".format(imgs_dir,len(self.ids),masks_dir))

    def __len__(self):
        return len(self.ids)

    @classmethod
    def process(cls, pil_img,pil_mask,preprocess=None):
        img = np.array(pil_img)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        mask = np.array(pil_mask)
        mask = cv2.cvtColor(mask, cv2.COLOR_RGB2BGR)
        if preprocess!= None:
            transformed = preprocess(image=img,mask=mask)
            img= transformed['image']
            mask= transformed['mask']

        img_trans = img.transpose((2, 0, 1))
        if img_trans.max() > 1:
            img_trans = img_trans / 255
        #mask_trans = mask.transpose((2,0,1))
        #print(mask.shape)
        Grayimg = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        ret, mask_trans = cv2.threshold(Grayimg, 12, 255,cv2.THRESH_BINARY)
        #cv2.imshow("sss",opencvImage)
        _ = {
            'image': torch.from_numpy(img_trans).type(torch.FloatTensor),
            'mask': torch.from_numpy(mask_trans).type(torch.FloatTensor).unsqueeze(0)
        }


        return _

    def __getitem__(self, i):
        idx = self.ids[i]
        temp = os.path.join(self.masks_dir,idx+self.mask_suffix+'.*')
        mask_file = glob.glob(temp)

        img_file = glob.glob(os.path.join(self.imgs_dir , idx + '.*'))

        assert len(mask_file) == 1, \
            f'Either no mask or multiple masks found for the ID {idx}: {mask_file}'
        assert len(img_file) == 1, \
            f'Either no image or multiple images found for the ID {idx}: {img_file}'
        mask = Image.open(mask_file[0])
        img = Image.open(img_file[0])

        assert img.size == mask.size, \
            f'Image and mask {idx} should be the same size, but are {img.size} and {mask.size}'
        #show_img(img)
        #img = self.process(img)
        #mask = self.process(mask)

        '''
        mask_trans = mask.transpose((2, 0, 1))
        if mask_trans.max() > 1:
            mask_trans = mask_trans / 255
        '''
        _ =  self.process(img,mask, self.preprocess)
        #print("mask shape",_['mask'].shape)
       # print("mask tensor",_['mask'])
        #print("image shape",_['image'].shape)
        return _
    
def train_net(net,
              device,
              epochs=10,
              batch_size=1,
              lr=0.001,
              val_percent=0.1,
              save_cp=True,
              criterion="dice",
              img_scale=512):

    #dataset = BasicDataset(train_dir, train_mask_dir, img_scale)
    dataset = BasicDataset(train_dir, train_mask_dir,AUGMENTATIONS_TRAIN,img_scale)
    dataset2 = BasicDataset(val_dir,val_mask_dir,AUGMENTATIONS_TEST)
    n_train=len(dataset)
    n_val = len(dataset2)
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
    val_loader = DataLoader(dataset2, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True, drop_last=True)

    writer = SummaryWriter(comment=f'LR_{lr}_BS_{batch_size}_SCALE_{img_scale}')
    global_step = 0
    # 基本信息
    logging.info(f'''Starting training:
        Epochs:          {epochs}
        Batch size:      {batch_size}
        Criterion:       {criterion}
        Learning rate:   {lr}
        Training size:   {len(dataset)}
        Validation size: {len(dataset2)}
        Checkpoints:     {save_cp}
        Device:          {device.type}
        Images scaling:  {img_scale}
    ''')
    #优化器
#    optimizer = optim.RMSprop(net.parameters(), lr=lr, weight_decay=1e-8, momentum=0.9)
    optimizer = optim.Adam(net.parameters(), lr=lr,  weight_decay=0.001)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min' if net.outc > 1 else 'max', patience=8)
    #if net.outc > 1:
    def dice_loss(input , target):
        input = torch.sigmoid(input)
        smooth = 0.00001
        iflat = input.view(-1)
        tflat = target.view(-1)
        intersection = (iflat * tflat).sum()
        return 1-((2.0*intersection + smooth)/(iflat.sum()+tflat.sum()+smooth))
    if  criterion=="bce":
        criterion = nn.BCEWithLogitsLoss()
    elif criterion ==  "cross":
        criterion == nn.CrossEntropyLoss()
    else:
        criterion = dice_loss

    for epoch in range(epochs):
        net.train()
        epoch_loss = 0
        with tqdm(total=n_train, desc=f'Epoch {epoch + 1}/{epochs}', unit='img') as pbar:
            for batch in train_loader:
                imgs = batch['image']
                true_masks = batch['mask']/255
                #true_masks =batch['mask']
                assert imgs.shape[1] == net.n_channels, \
                    f'Network has been defined with {net.n_channels} input channels, ' \
                    f'but loaded images have {imgs.shape[1]} channels. Please check that ' \
                    'the images are loaded correctly.'

                imgs = imgs.to(device=device, dtype=torch.float32)
                #mask_type = torch.float32 if net.outc == 1 else torch.long
                mask_type = torch.float32
                true_masks = true_masks.to(device=device, dtype=mask_type)
                #print("true mask shape: %s " % true_masks.shape)
                masks_pred = net(imgs)
                print(masks_pred)
                #print("predict mask shape:%s" % mask_pred.shape)
                loss = criterion(masks_pred,true_masks)
                epoch_loss += loss.item()
                writer.add_scalar('Loss/train', loss.item(), global_step)

                pbar.set_postfix(**{'loss (batch)': loss.item()})

                optimizer.zero_grad()
                loss.backward()
                nn.utils.clip_grad_value_(net.parameters(), 0.1)
                optimizer.step()

                pbar.update(imgs.shape[0])
                global_step += 1
                if global_step % (n_train // (10 * batch_size)) == 0:
                    logging.info('item_loss: {}'.format(epoch_loss))
                    for tag, value in net.named_parameters():
                        tag = tag.replace('.', '/')
                        writer.add_histogram('weights/' + tag, value.data.cpu().numpy(), global_step)
                        writer.add_histogram('grads/' + tag, value.grad.data.cpu().numpy(), global_step)
                    val_score = eval_net(net, val_loader, device)
                    scheduler.step(val_score)
                    writer.add_scalar('learning_rate', optimizer.param_groups[0]['lr'], global_step)
                    logging.info('lr: {}'.format(optimizer.param_groups[0]['lr']))
                    if net.n_classes > 1:
                        logging.info('Validation cross entropy: {}'.format(val_score))
                        writer.add_scalar('Loss/test', val_score, global_step)
                    else:
                        logging.info('Validation Dice Coeff: {}'.format(val_score))
                        writer.add_scalar('Dice/test', val_score, global_step)

                    writer.add_images('images', imgs, global_step)
                    if net.n_classes == 1:
                        writer.add_images('masks/true', true_masks, global_step)
                        writer.add_images('masks/pred', torch.sigmoid(masks_pred) > 0.5, global_step)

        if save_cp:
            try:
                os.mkdir(dir_checkpoint)
                logging.info('Created checkpoint directory')
            except OSError:
                pass
            torch.save(net.state_dict(),
                       dir_checkpoint + f'{args.net}batch{args.batchsize}scale{args.scale}epoch{epoch + 1}.pth')
            logging.info(f'Checkpoint {epoch + 1} saved !')

    writer.close()


def get_args():
    parser = argparse.ArgumentParser(description='Train the UNet on images and target masks',
                                     formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument('-e', '--epochs', metavar='E', type=int, default=20,
                        help='Number of epochs', dest='epochs')
    parser.add_argument('-n', '--net' ,dest="net",type=str,default="unet",
                        help="choose net like resnet")
    parser.add_argument('-b', '--batch-size', metavar='B', type=int, nargs='?', default=4,
                        help='Batch size', dest='batchsize')
    parser.add_argument('-l', '--learning-rate', metavar='LR', type=float, nargs='?', default=0.0001,
                        help='Learning rate', dest='lr')
    parser.add_argument('-f', '--load', dest='load', type=str, default=False,
                        help='Load model from a .pth file')
    parser.add_argument('-s', '--scale', dest='scale', type=float, default=256,
                        help='Downscaling factor of the images')
    parser.add_argument('-v', '--validation', dest='val', type=float, default=10.0,
                        help='Percent of the data that is used as validation (0-100)')
    parser.add_argument('-c', '--criterion', dest='criterion',type=str,default='dice',
                        help = "criterion like bce crossentropyloss, dice_loss")

    return parser.parse_args()


if __name__ == '__main__':
    logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
    args = get_args()
    #device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    device =torch.device("cuda:1")
    logging.info(f'Using device {device}')

    # Change here to adapt to your data
    # n_channels=3 for RGB images
    # n_classes is the number of probabilities you want to get per pixel
    #   - For 1 class and background, use n_classes=1
    #   - For 2 classes, use n_classes=1
    #   - For N > 2 classes, use n_classes=N
    args.net="nested_unet"
    args.load="./check_point/nested_unetbatch4scale512.0epoch15.pth"
    net =  model.choose_net(args.net)
    net = net(in_channel=3,out_channel=1)
    logging.info(f'Network:\n'
                f'\t{net.n_channels} input channels\n'
                 f'\t{net.n_classes} output channels (classes)\n'
                 f'\t{"Bilinear" if args.load else "Transposed conv"} upscaling')

    if args.load:
        net.load_state_dict(
            torch.load(args.load, map_location=device)
            )
        logging.info(f'Model loaded from {args.load}')
    

    net.to(device=device)
    # faster convolutions, but more memory
    # cudnn.benchmark = True

    try:
        train_net(net=net,
                  epochs=args.epochs,
                  batch_size=args.batchsize,
                  lr=args.lr,
                  criterion=args.criterion,
                  device=device,
                  img_scale=args.scale,
                  val_percent=args.val / 100)
    except KeyboardInterrupt:
        torch.save(net.state_dict(), args.net+args.batchsize+'INTERRUPTED.pth')
        logging.info('Saved interrupt')
        try:
            sys.exit(0)
        except SystemExit:
            os._exit(0)




INFO: Using device cuda:1
INFO: Network:
	3 input channels
	1 output channels (classes)
	Bilinear upscaling
INFO: Model loaded from ./check_point/nested_unetbatch4scale512.0epoch15.pth
INFO: Creating dataset with 297 examples
INFO: Creating dataset with 33 examples
INFO: Starting training:
        Epochs:          20
        Batch size:      4
        Criterion:       dice
        Learning rate:   0.0001
        Training size:   297
        Validation size: 33
        Checkpoints:     True
        Device:          cuda
        Images scaling:  256
    
Epoch 1/20:   0%|          | 0/297 [00:05<?, ?img/s, loss (batch)=0.569]

tensor([[[[-1.5669, -1.6165, -1.6652,  ..., -2.0683, -2.0758, -1.8660],
          [-1.4900, -1.5181, -1.6512,  ..., -2.4038, -2.3026, -2.0326],
          [-1.6204, -1.7259, -1.8472,  ..., -2.5407, -2.4844, -2.0816],
          ...,
          [-1.9034, -2.3368, -2.4878,  ..., -2.7091, -2.6414, -2.1667],
          [-1.9258, -2.3424, -2.5089,  ..., -2.6641, -2.5458, -2.1319],
          [-1.7190, -1.9431, -2.0313,  ..., -2.1147, -2.0402, -1.8922]]],


        [[[-1.3340, -1.2455, -1.2631,  ..., -0.9977, -0.9868, -1.0472],
          [-1.1205, -1.0966, -1.1467,  ..., -1.0637, -1.0589, -0.9070],
          [-1.1328, -1.0857, -1.1352,  ..., -0.9262, -0.9675, -0.8514],
          ...,
          [-1.0224, -1.1697, -1.2067,  ..., -0.9512, -0.9753, -0.8490],
          [-1.0326, -1.1514, -1.1927,  ..., -1.0540, -1.0503, -0.9023],
          [-1.0804, -0.9768, -0.9606,  ..., -0.8599, -0.8582, -0.9247]]],


        [[[-1.2968, -1.2132, -1.2480,  ..., -1.3859, -1.3458, -1.3877],
          [-1.1038, -1.325

Epoch 1/20:   1%|▏         | 4/297 [00:26<07:34,  1.55s/img, loss (batch)=0.895]

tensor([[[[-1.8238, -2.0363, -2.0839,  ..., -2.1562, -2.1503, -1.9340],
          [-1.8638, -2.2307, -2.4285,  ..., -2.6117, -2.5280, -2.1402],
          [-1.8730, -2.2757, -2.4799,  ..., -2.7378, -2.6922, -2.1867],
          ...,
          [-1.6506, -1.7152, -1.8073,  ..., -0.5690, -0.6569, -0.7037],
          [-1.8451, -2.0662, -2.0762,  ..., -0.6954, -0.7251, -0.8627],
          [-1.7416, -1.9192, -1.9233,  ..., -0.7317, -0.8150, -1.1031]]],


        [[[-1.4636, -1.3715, -1.3449,  ..., -1.9568, -1.9950, -1.8284],
          [-1.2011, -1.0634, -1.0873,  ..., -2.2277, -2.1895, -1.9452],
          [-1.1330, -1.0785, -1.1422,  ..., -2.2824, -2.2853, -1.9493],
          ...,
          [-1.7616, -1.8327, -1.8062,  ..., -1.5383, -1.6853, -1.7696],
          [-1.8424, -1.9952, -2.0069,  ..., -1.8101, -1.8599, -1.8258],
          [-1.7121, -1.8417, -1.8652,  ..., -1.7566, -1.7531, -1.7367]]],


        [[[-1.8321, -2.0564, -2.1053,  ..., -2.1632, -2.1560, -1.9375],
          [-1.8807, -2.275

Epoch 1/20:   3%|▎         | 8/297 [00:27<18:12,  3.78s/img, loss (batch)=0.715]

...

Epoch 7/20:  70%|███████   | 208/297 [11:45<47:59, 32.36s/img, loss (batch)=0.805]

tensor([[[[-1.8125, -2.1775, -2.3026,  ..., -1.6911, -1.6570, -1.5679],
          [-1.8552, -2.2729, -2.4733,  ..., -1.6357, -1.5386, -1.5274],
          [-1.9020, -2.3345, -2.4908,  ..., -1.7296, -1.5494, -1.5122],
          ...,
          [-1.9682, -2.3959, -2.4931,  ..., -2.6501, -2.7120, -2.4020],
          [-1.9656, -2.4168, -2.5276,  ..., -2.6426, -2.6430, -2.3509],
          [-1.6980, -2.0056, -2.1214,  ..., -2.1859, -2.1421, -1.9716]]],


        [[[-1.7882, -2.1344, -2.2586,  ..., -2.4069, -2.3637, -2.0114],
          [-1.8219, -2.2073, -2.4137,  ..., -2.6596, -2.6328, -2.3028],
          [-1.8577, -2.2664, -2.4435,  ..., -2.6933, -2.7364, -2.4128],
          ...,
          [-0.8715, -0.7608, -0.4319,  ..., -0.0285, -0.3111, -0.4421],
          [-1.0802, -0.9584, -0.6954,  ..., -0.3413, -0.5783, -0.6227],
          [-1.1329, -0.9296, -0.7375,  ..., -0.3925, -0.5286, -0.7012]]],


        [[[-1.7980, -2.1560, -2.2832,  ..., -0.2314, -0.4068, -0.5717],
          [-1.8370, -2.242

Epoch 7/20:  71%|███████▏  | 212/297 [11:46<32:11, 22.73s/img, loss (batch)=0.507]

tensor([[[[-2.0794, -2.5615, -2.5946,  ..., -2.3163, -2.3456, -2.0298],
          [-2.2254, -2.7667, -2.7813,  ..., -2.5250, -2.5737, -2.3042],
          [-2.1969, -2.6990, -2.6977,  ..., -2.5058, -2.5965, -2.3442],
          ...,
          [-1.3297, -1.2859, -1.1724,  ..., -2.3315, -2.2860, -2.0886],
          [-1.4496, -1.4033, -1.2975,  ..., -2.4216, -2.3605, -2.1407],
          [-1.3801, -1.3142, -1.1961,  ..., -2.0456, -1.9881, -1.8617]]],


        [[[-1.3041, -1.1465, -1.0114,  ..., -0.6104, -0.7940, -0.9662],
          [-1.0931, -1.1676, -1.0035,  ..., -0.4392, -0.7206, -0.7535],
          [-0.9580, -1.0234, -0.7819,  ..., -0.0190, -0.3747, -0.5277],
          ...,
          [-1.5415, -1.7301, -1.7457,  ...,  0.0672, -0.2743, -0.4609],
          [-1.6045, -1.7039, -1.7147,  ..., -0.3678, -0.6225, -0.7246],
          [-1.4123, -1.4257, -1.3751,  ..., -0.4810, -0.6339, -0.8346]]],


        [[[-2.0295, -2.4100, -2.3774,  ..., -1.9365, -1.9262, -1.8069],
          [-2.1353, -2.527

Epoch 7/20:  73%|███████▎  | 216/297 [11:47<21:35, 15.99s/img, loss (batch)=0.507]INFO: item_loss: 35.8653559088707
